## Project 5 : Classification

## Instructions

### Description

Practice classification on the Titanic dataset.

### Grading

For grading purposes, we will clear all outputs from all your cells and then run them all from the top.  Please test your notebook in the same fashion before turning it in.

### Submitting Your Solution

To submit your notebook, first clear all the cells (this won't matter too much this time, but for larger data sets in the future, it will make the file smaller).  Then use the File->Download As->Notebook to obtain the notebook file.  Finally, submit the notebook file on Canvas.


In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Introduction

On April 15, 1912, the largest passenger liner ever made collided with an iceberg during her maiden voyage. When the Titanic sank it killed 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships. One of the reasons that the shipwreck resulted in such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others.

Intro Videos: 
https://www.youtube.com/watch?v=3lyiZMeTKIo
and
https://www.youtube.com/watch?v=ItjXTieWKyI 

The `titanic_data.csv` file contains data for `887` of the real Titanic passengers. Each row represents one person. The columns describe different attributes about the person including whether they survived (`0=No`), their age, their passenger-class (`1=1st Class, Upper`), gender, and the fare they paid (£s*). For more on the currency: http://www.statisticalconsultants.co.nz/blog/titanic-fare-data.html

We are going to try to see if there are correlations between the feature data provided (find a best subset of features) and passenger survival.

### Problem 1: Load and understand the data (35 points)

#### Your task (some of this is the work you completed for L14 - be sure to copy that work into here as needed)
Conduct some preprocessing steps to explore the following and provide code/answers in the below cells:
1. Load the `titanic_data.csv` file into a pandas dataframe
2. Explore the data provided (e.g., looking at statistics using describe(), value_counts(), histograms, scatter plots of various features, etc.) 
3. What are the names of feature columns that appear to be usable for learning?
4. What is the name of the column that appears to represent our target?
5. Formulate a hypothesis about the relationship between given feature data and the target
6. How did Pclass affect passenngers' chances of survival?
7. What is the age distribution of survivors?

In [ ]:
# Step 1. Load the `titanic_data.csv` file into a pandas dataframe
titanic = pd.read_csv("titanic_data.csv")

In [ ]:
# Step 2. Explore the data provided (e.g., looking at statistics using describe(), value_counts(), histograms, scatter plots of various features, etc.) 
#pd.get_dummies(titanic['Sex'])
print(titanic.describe())
print("\n\nAge\n", titanic['Age'].value_counts())
print(titanic.columns)
plt.figure()
titanic.hist(figsize=(20,20), layout=(6,1))
plt.show()
plt.figure()
plt.scatter(titanic['Siblings/Spouses Aboard'], titanic['Survived'])
plt.title("Siblings/Spouses Aboard VS Survived")
plt.show()
plt.scatter(titanic['Fare'], titanic['Survived'])
plt.title("Fare VS Survived")
plt.show()
titanic.head()


---

**Edit this cell to provide answers to the following steps:**

---

Step 3. What are the names of feature columns that appear to be usable for learning?
    
    'Survived', 'Pclass', 'Sex' (concated with 'male' and 'female'), 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare'
    
Step 4. What is the name of the column that appears to represent our target?

    'Survived'
    
Step 5. Formulate a hypothesis about the relationship between given feature data and the target
    
       The 'Age', gender, and relationship with others (sibling/spouse/parent) on board effect the outcome of survival. The younger you are, being a female, and having others on board (Parents, Children, Siblings, Spouses) increase the chance of survival.

In [ ]:
#Step 6. How did Pclass affect passenngers' chances of survival?

'''The higher the class, the more likely you were to survive the titanic. First class died more than survived, Second class
    survived about the same amount as the died, and Third class was much more likely to survive'''

#Show your work with a bar plot, dataframe selection, or visual of your choice.
survived = titanic[titanic['Survived'] == 0]
died = titanic[titanic['Survived'] == 1]


fig, ax = plt.subplots()

a_heights, a_bins = np.histogram(survived['Pclass'])
b_heights, b_bins = np.histogram(died['Pclass'], bins=a_bins)

width = 0.2

ax.bar(a_bins[:-1], a_heights, width=width, facecolor='cornflowerblue')
ax.bar(b_bins[:-1]+width, b_heights, width=width, facecolor='seagreen')
plt.ylabel("People")
plt.xlabel("Pclass")
plt.title("Survived VS Pclass")
plt.legend(('Survived', 'Died'))
plt.show()


In [ ]:
#Step 7. What is the age distribution of survivors?
#Show your work with a dataframe operation and/or histogram plot.
plt.figure()
plt.hist(titanic['Age'])
plt.xlabel("Age")
plt.ylabel("Number of People")
plt.title("Age Distribution")
plt.show()

### Problem 2: transform the data (10 points)
The `Sex` column is categorical, meaning its data are separable into groups, but not numerical. To be able to work with this data, we need numbers, so you task is to transform the `Sex` column into numerical data with pandas' `get_dummies` feature and remove the original categorical `Sex` column.

In [ ]:
#Done in Problem one (first line)
dummy = pd.get_dummies(titanic['Sex'])
titanic = pd.concat([titanic, dummy], axis=1)
titanic.head()

### Problem 3: Classification (30 points)
Now that the data is transformed, we want to run various classification experiments on it. The first is `K Nearest Neighbors`, which you will conduct by:

1. Define input and target data by creating lists of dataframe columns (e.g., inputs = ['Pclass', etc.)
2. Split the data into training and testing sets with `train_test_split()`
3. Create a `KNeighborsClassifier` using `5` neighbors at first (you can experiment with this parameter)
4. Train your model by passing the training dataset to `fit()`
5. Calculate predicted target values(y_hat) by passing the testing dataset to `predict()`
6. Print the accuracy of the model with `score()`

** Note: If you get a python warning as you use the Y, trainY, or testY vector in some of the function calls about "DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, )", you can look up how to use trainY.values.ravel() or trainY.values.flatten() or another function, etc.

In [ ]:
inputs = ('female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard')
target = ('Survived')

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
train, test = train_test_split(titanic)
train.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 5
model = KNeighborsClassifier(k)
model.fit(train[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], train['Survived'])
predicted = model.predict(test[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']])
print("Accuracy:\t", model.score((test[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']]), test['Survived']))

### Problem 4: Cross validation, classification report (15 points)
- Using the concepts from the 17-model_selection slides and the [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function from scikit-learn, estimate the f-score ([`f1-score`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score) (you can use however many folds you wish). To get `cross_val_score` to use `f1-score` rather than the default accuracy measure, you will need to set the `scoring` parameter and use a scorer object created via [`make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html#sklearn.metrics.make_scorer).  Since this has a few parts to it, let me just give you that parameter: ```scorerVar = make_scorer(f1_score, pos_label=1)```

- Using the concepts from the end of the 14-classification slides, output a confusion matrix.

- Also, output a classification report [`classification_report`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) from sklearn.metrics showing more of the metrics: precision, recall, f1-score for both of our classes.

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, classification_report, make_scorer
from sklearn import model_selection

f1 = f1_score(test['Survived'], predicted)
print("f1_score:\t", f1)
scorerVar = make_scorer(f1_score, pos_label=1)
cross_score = cross_val_score(model, train[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], train['Survived'], scoring= scorerVar, cv = 5)
print("\n\nCross Val Score with f1_scorer:\n", cross_score)
print("\n\nConfusion Matrix:\n",confusion_matrix(test['Survived'], predicted))
print("\n\nClassification Report:\n", classification_report(test['Survived'], predicted))

### Problem 5: Logistic Regression (15 points)

Now, repeat the above experiment using the [`LogisticRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model in scikit-learn, and output:

- The fit accuracy (using the `score` method of the model)
- The f-score (using the [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function)
- The confusion matrix
- The precision, recall, and f-measure for the 1 class (you can just print the results of the [`classification_report`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) function from sklearn.metrics)

In [ ]:
from sklearn.linear_model import LogisticRegression
#create a model object
model = LogisticRegression()

#train our model
model.fit(train[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], train['Survived'])
predicted = model.predict(test[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']])

#evaluate the model 
score = model.score(test[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], test['Survived'])
print("Score:\t", score)

# # #setup to get f-score and cv
cross_score = cross_val_score(model, train[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], train['Survived'], scoring= scorerVar, cv = 5)
print("\n\nCross Val using f1_score:\n", cross_score)
      
# # # #confusion matrix
print("\n\nConfusion Matrix:\n", confusion_matrix(test['Survived'], predicted))
      
# # # #classification report
report = classification_report(test['Survived'], predicted)
print("\n\nClassification Report:\n", report)

### Problem 6: Support Vector Machines (15 points)
Now, repeat the above experiment using the using a Support Vector classifier [`SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) with default parameters (RBF kernel) model in scikit-learn, and output:

- The fit accuracy (using the `score` method of the model)
- The f-score (using the [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function)
- The confusion matrix
- The precision, recall, and f-measure for the 1 class (you can just print the results of the [`classification_report`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) function from sklearn.metrics)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

#create a model object
model = SVC(kernel='linear', C=10)

#train our model
model.fit(train[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], train['Survived'])
predicted = model.predict(test[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']])

#evaluate the model 
score = model.score(test[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], test['Survived'])
print("Score:\t", score)

#setup to get f-score and cv
cross_score = cross_val_score(model, train[['female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard']], train['Survived'], scoring= scorerVar, cv = 5)
print("\n\nCross Val using f1_score:\n", cross_score)

#confusion matrix
print("\n\nConfusion Matrix:\n", confusion_matrix(test['Survived'], predicted))

#classification report
report = classification_report(test['Survived'], predicted)
print("\n\nClassification Report:\n", report)

### Problem 7: Comparision and Discussion (5 points)
Edit this cell to provide a brief discussion (3-5 sentances at most):
1. What was the model/algorithm that performed best for you?

Using the parameters I did, Logistic Regression and Support Vector Machines preformed best with nearly identical results.

2. What feaures and parameters were used to achieve that performance?

I used ('female', 'male', 'Age', 'Siblings/Spouses Aboard','Parents/Children Aboard') for the labels and ('Survived') for the target.

3. What insights did you gain from your experimentation about the predictive power of this dataset and did it match your original hypothesis about the relationship between given feature data and the target?

I learned that this dataset is okay at predicting with the given parameters. The scores produced aren't fantastic, but with the parameters used, the models preformed better than when run with all parameter (ran with all parameters to see stats before changing to hypothesis). The models seemed to model the hypothesis to a good degree (scores ~ .80), but not an excellent model.

### Questionnaire
1) How long did you spend on this assignment?
<br>
3 hours
<br>
2) What did you like about it? What did you not like about it?
<br>
I liked seeing how the different models preformed, but I thought that there were many concepts that weren't well covered in the notebooks provided.
<br>
3) Did you find any errors or is there anything you would like changed?
<br>
I didn't find any errors, but on topics not in the notebooks, more than just the page documentation would be more helpful.
<br>